In [1]:
### 生成输出目录的模块
import os
from datetime import datetime

def generate_output(disturb: bool, filename):
    # Generate the folder name with the current date
    now = datetime.now()
    if disturb:
        folder_name = f"Graph_Rmv_Stoch_{now.strftime('%Y-%m-%d-%H')}"
    else:
        folder_name = f"Graph_Rmv_Btwn_{now.strftime('%Y-%m-%d-%H')}"

    # Create the output directory if it doesn't exist
    output_dir = os.path.join(os.getcwd(), folder_name)
    os.makedirs(output_dir, exist_ok=True)

    # Construct the full file path
    file_path = os.path.join(output_dir, filename)

    return file_path

In [2]:
import os
from datetime import datetime

import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

n = 1000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1  # Power-law exponent for the community size distribution
mu = 0.1  # Mixing parameter
avg_deg = 25  # Average Degree
max_deg = int(0.1 * n)  # Max Degree
min_commu = 60  # Min Community Size
max_commu = int(0.1 * n)  # Max Community Size

G0 = LFR_benchmark_graph(
    n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg, min_community=min_commu, max_community=max_commu,
    seed=7
)

# Remove multi-edges and self-loops from G0
G0 = nx.Graph(G0)
selfloop_edges = list(nx.selfloop_edges(G0))
G0.remove_edges_from(selfloop_edges)

# Generate a timestamp for the output file names
now = datetime.now()

# Write the graph's edge list to a file
edge_output_file = f"edgelist_{now.strftime('%Y-%m-%d-%H')}.txt"
nx.write_edgelist(G0, edge_output_file, delimiter=' ', data=False)

import numpy as np
# Get intrinsic membership and save it to a file
intrinsic_communities = {frozenset(G0.nodes[v]["community"]) for v in G0}
intrinsic_membership = np.empty(G0.number_of_nodes(), dtype=int)
for node in range(G0.number_of_nodes()):
    for index, inner_set in enumerate(intrinsic_communities):
        if node in inner_set:
            intrinsic_membership[node] = index
            break

membership_output_file = f"membership_{now.strftime('%Y-%m-%d-%H')}.txt"
np.savetxt(membership_output_file, intrinsic_membership, delimiter=' ', fmt='%d')

# Get betweenness centrality and save it to a file
betweenness = list(nx.betweenness_centrality(G0).values())
betweenness_output_file = f"betweenness_{now.strftime('%Y-%m-%d-%H')}.txt"
np.savetxt(betweenness_output_file, betweenness, delimiter=' ')


In [11]:
### 读取存储的图的相关信息
### 从文件中读入内容并转化成 边列表
with open('edgelist_2023-07-11-17.txt', 'r') as file:
    lines = file.readlines()

### Process the lines and create a list of number pairs
edge_list = []
for line in lines:
    pair = tuple(map(int, line.strip().split()))
    edge_list.append(pair)

### Print the list of number pairs
#print(edge_list)

import networkx as nx

### 新建一个图 
G0 = nx.Graph()

### 向图添加边
G0.add_edges_from(edge_list)

### 记录有几个结点
N = G0.number_of_nodes()

### Load Community Info
import numpy as np

membership_list = "membership_2023-07-11-17.txt"
intrinsic_membership = np.loadtxt(membership_list, dtype=int)

btwn_file = "betweenness_2023-07-11-17.txt"
betweenness = np.loadtxt(btwn_file)

In [3]:
### 生成删除顶点的顺序 模块
import random
import networkx as nx

def nodes_sample(G, disturb: bool, number_of_nodes: int, percent, betweenness):
    graph = G.copy()
    sample_size = int(number_of_nodes*percent)
    if disturb:
        removed_nodes = random.sample(range(number_of_nodes), sample_size)
        graph.remove_nodes_from(removed_nodes)
        if nx.is_connected(graph):
            return removed_nodes
    else: 
        removed_nodes = random.choices(range(number_of_nodes), betweenness, k=sample_size)
        graph.remove_nodes_from(removed_nodes)
        if nx.is_connected(graph):
            return removed_nodes        
        
#nodes_sample(G0, N, 0.8)

import numpy as np
import json

def generate_remove_list(G, disturb: bool, number_of_nodes, betweenness):
    remove_procedure = []
    for percent in np.arange(0.05, 0.86, 0.05):
        ls = []
        while len(ls)<50:
            temp = nodes_sample(G = G, disturb = disturb, number_of_nodes = number_of_nodes, percent = percent, betweenness = betweenness)
            if not temp is None:
                ls.append(temp)
        remove_procedure.append(ls)
    filename = generate_output(disturb, "remove_procedure.txt")
    with open(filename, 'w') as file:
        json.dump(remove_procedure, file)
    return remove_procedure

In [4]:
### 读取存储的删除顶点的顺序信息
ls = generate_remove_list(G0, True, G0.number_of_nodes(), betweenness)
with open('Graph_Rmv_Stoch_2023-07-11-18/remove_procedure.txt', 'r') as file:
    loaded_data = json.load(file)
loaded_data == ls

True